<a href="https://colab.research.google.com/github/moyeed/corelation_between_jobs_and_research_papers/blob/main/Scraping_Indeed_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Scraping Indeed Jobs Data through beutifulSoup and Requests
## Developers : 
> * Mohammed Abdul Moyeed
> * Omer Bin Ali Bajubair


##Adding Import statements

In [ ]:
from bs4 import BeautifulSoup
import requests
from random import random
from time import sleep
import csv
import pandas as pd

## Defining required functions

In [ ]:
def generate_url(job_title= "", job_location = "Dekalb,IL",fromage = 7):
    url_template = "https://www.indeed.com/jobs?q={}&l={}&fromage={}"
    url = url_template.format(job_title, job_location,fromage)
    return url

def request_jobs_from_indeed(url):
    headers = {
        'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,'
                  'application/signed-exchange;v=b3;q=0.9',
        'accept-encoding': 'gzip, deflate, br',
        'accept-language': 'en-US,en;q=0.9',
        'cache-control': 'max-age=0',
        'sec-fetch-dest': 'document',
        'sec-fetch-mode': 'navigate',
        'sec-fetch-site': 'none',
        'sec-fetch-user': '?1',
        'upgrade-insecure-requests': '1',
        'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) '
                      'Chrome/87.0.4280.67 Safari/537.36 Edg/87.0.664.47 '
    }
    response = requests.get(url) #, headers = headers
    # print(response.text)
    if response.status_code == 200:
        return response.text
    else:
        return None

def collect_job_cards_from_page(html,tag,tagName):
    soup = BeautifulSoup(html, 'html.parser')
    # cards = soup.find_all('a',  "tapItem") 
    cards = soup.find_all(tag,  tagName) 
    return cards, soup


def sleep_for_random_interval():
    seconds = random() * 10
    sleep(seconds)


def find_next_page(soup):
    try:
        pagination = soup.find("a", {"aria-label": "Next"}).get("href")
        return "https://www.indeed.com" + pagination
    except AttributeError:
        return None

def extract_job_card_data(card):
    atag = card.get('href')
    id = card.get('href')
    iframe = card.get('')
    try:
        job_title = card.find('td','resultContent').find('div', 'heading4 color-text-primary singleLineTitle tapItem-gutter').find("h2","jobTitle jobTitle-color-purple jobTitle-newJob").text.strip()
    except AttributeError:
        job_title = ''
    try:
        company = card.find('span', 'companyName').text.strip()
    except AttributeError:
        company = ''
    try:
        location = card.find('div', 'companyLocation').text.strip()#get('data-rc-loc')
    except AttributeError:
        location = ''
    try:
        job_summary = card.find('div', 'job-snippet').text.strip()
    except AttributeError:
        job_summary = ''
    try:
        post_date = card.find('span', 'date').text.strip()
    except AttributeError:
        post_date = ''
    try:
        salary = card.find('div', 'metadata salary-snippet-container').find('div','attribute_snippet').text.strip()
    
    except AttributeError:
        salary = ''
    job_url =  'https://www.indeed.com'+atag
    
    return job_title, company, location, job_summary, salary, post_date, job_url

def get_parsed_full_desc(p_ul_elements):
  if(len(p_ul_elements)>0):

    p_elements = p_ul_elements[0].find_all('p')
    li_elements = p_ul_elements[0].find_all("li")
    full_desc = "";
    for each_para in p_elements:
      text = each_para.text.strip()
      full_desc += ((text+"\n") if (len(text.split(" "))>10) else (""))
    
    for each_para in li_elements:
      text = each_para.text.strip()
      full_desc += ((text+"\n") if (len(text.split(" "))>3) else (""))
    return full_desc
  else:
    print("len of p_ul_elements was zero, it is printted below")
    print(p_ul_elements)
    return  ""


def save_record_to_csv(record, filepath, create_new_file=False):
    """Save an individual record to file; set `new_file` flag to `True` to generate new file"""
    header = ["JobTitle", "Company", "Location", "Summary", "Salary", "PostDate", "JobUrl"]
    if create_new_file:
        with open(filepath, mode='w', newline='', encoding='utf-8') as f:
            writer = csv.writer(f)
            writer.writerow(header)
    else:
        with open(filepath, mode='a+', newline='', encoding='utf-8') as f:
            writer = csv.writer(f)
            writer.writerow(record)

print(" functions loaded")

 functions loaded


In [ ]:
job_title = 'Robotics'
job_location = '' #Chicago, IL
filepath = 'Robo_jobs_USA(proper links).csv'
days_old = 30 # 2-years => 730
unique_jobs = set()  # track job urls to avoid collecting duplicate records
print("Starting to scrape indeed for `{}` in `{}`".format(job_title, job_location))
url = generate_url(job_title = job_title, job_location = job_location, fromage=days_old)
save_record_to_csv(None, filepath, create_new_file=True)
fetch_count = 0;

while True:
    print(url)
    html = request_jobs_from_indeed(url)
    if not html:
        break
    cards, soup = collect_job_cards_from_page(html,'a',  "tapItem")
    print("total jobs fetched :: ", len(cards))
    fetch_count += len(cards)
    for card in cards:
        record = extract_job_card_data(card)
        if not record[-1] in unique_jobs:
            save_record_to_csv(record, filepath)
            unique_jobs.add(record[-1])
    sleep_for_random_interval()
    url = find_next_page(soup)
    if not url:
        break

print('Finished collecting {:,d} job postings.'.format(len(unique_jobs)))
print("fetched count :: ", fetch_count)

Starting to scrape indeed for `Robotics` in ``
1
23785
https://www.indeed.com/jobs?q=Robotics&l=&fromage=30
total jobs fetched ::  15
https://www.indeed.com/jobs?q=Robotics&fromage=30&start=10
total jobs fetched ::  15


KeyboardInterrupt: ignored

In [ ]:
titles = ["NLP","Computer Vision", "Machine Learning", "Robotics"] #
data = {}
title_dict = {}
timeFrame = 365  #1460
year = 12
while timeFrame > 0:
  title_dict = {}
  for title in titles:
    job_title = title
    job_location = '' #Chicago, IL
    filepath = 'jobs_count.csv'
    print(timeFrame)
    days_old = timeFrame # 2-years => 730
    print("Starting to scrape indeed for `{}` in `{}`".format(job_title, job_location))
    url = generate_url(job_title = job_title, job_location = job_location, fromage=days_old)
    print(url)
    jd_card, _ = collect_job_cards_from_page(request_jobs_from_indeed(url),'div',  "searchCount-a11y-contrast-color")
    print(jd_card)
    jd_count = (jd_card[0].find_all("div")[0].text.strip().split("of"))[1].split(" ")[1]
    title_dict[title] = int(jd_count.strip().replace(",", ""))
    print(int(jd_count.strip().replace(",", "")))
  data[year]  = title_dict
  timeFrame -= 30 #365
  year -= 1
    # print(len(jd_card))
    # print(int(jd_count.strip().replace(",", "")))

365
Starting to scrape indeed for `NLP` in ``
https://www.indeed.com/jobs?q=NLP&l=&fromage=365
[<div class="searchCount-a11y-contrast-color" id="searchCount">
<div id="searchCountPages">
                    Page 1 of 13,211 jobs</div>
<div class="serp-relevance-explanation"><button aria-haspopup="true" aria-label="help" class="serp-relevance-explanation-helpIcon serp-helpIcon" type="button"><svg height="16" width="16" xmlns="http://www.w3.org/2000/svg" xmlns:xlink="http://www.w3.org/1999/xlink"><defs><lineargradient id="helpIcon-a" x1="50%" x2="50%" y1="0%" y2="100%"><stop offset="0%" stop-color="#FFF" stop-opacity=".5"></stop><stop offset="100%" stop-opacity=".5"></stop></lineargradient><lineargradient id="helpIcon-b" x1="50%" x2="50%" y1="0%" y2="100%"><stop offset="0%" stop-opacity=".5"></stop><stop offset="100%" stop-opacity=".5"></stop></lineargradient><path d="M7.1537 3.391C8.373 3.4665 9.3466 4.44 9.4223 5.6594 9.4886 6.7088 8.8736 7.6823 7.9 8.0702c-.1413.0563-.2358.1796-.2358.

In [ ]:
from pprint import pprint
pprint(data)

{0: {'Computer Vision': 13193,
     'Machine Learning': 11431,
     'NLP': 1116,
     'Robotics': 8155},
 1: {'Computer Vision': 46082,
     'Machine Learning': 29335,
     'NLP': 4539,
     'Robotics': 26901},
 2: {'Computer Vision': 66732,
     'Machine Learning': 41589,
     'NLP': 7209,
     'Robotics': 36971},
 3: {'Computer Vision': 78026,
     'Machine Learning': 49766,
     'NLP': 9133,
     'Robotics': 43486},
 4: {'Computer Vision': 108129,
     'Machine Learning': 71433,
     'NLP': 13204,
     'Robotics': 61530},
 5: {'Computer Vision': 108129,
     'Machine Learning': 71433,
     'NLP': 13204,
     'Robotics': 61530},
 6: {'Computer Vision': 108129,
     'Machine Learning': 71433,
     'NLP': 13204,
     'Robotics': 61530},
 7: {'Computer Vision': 108129,
     'Machine Learning': 71433,
     'NLP': 13204,
     'Robotics': 61530},
 8: {'Computer Vision': 108129,
     'Machine Learning': 71433,
     'NLP': 13204,
     'Robotics': 61530},
 9: {'Computer Vision': 108129,
     

In [ ]:
df = pd.DataFrame()
df = pd.read_csv('/content/Robo_jobs_USA(proper links).csv')

In [ ]:
def extract_full_desc(html):
  jd_card, _ = collect_job_cards_from_page(html,'div',  "jobsearch-jobDescriptionText")
  full_jd = get_parsed_full_desc(jd_card)
  return full_jd

df["full_desc"] = [" "]*len(df)
for data in df.iterrows():
  html = request_jobs_from_indeed(data[1]['JobUrl'])
  if(html == None):
    continue
  full_desc = extract_full_desc(html)
  print(data[1]['JobUrl'])
  df.iloc[data[0]]["full_desc"] = full_desc

https://www.indeed.com/pagead/clk?mo=r&ad=-6NYlbfkN0CU3GZ4cyNO5mMUn6Yvwwiat6CQGslVKhnqfG35EkNYkk0EC0yF2zqbhD-8rNxvYw-qYzibEAIWtCzCrPKNzJdYNfZsSlozg-Ry3cn3PyZi26vVR63rCSklVcOocj5xm1kWZ9CmgKF-BxO70rsbbtEs7boio2Jm5u9Z6fUv-coldmLLErDfCD0B_D4-uMGkq6wZSFm8SRoL4ADZsVgVKLdMQIgBSw-pyFTbvPu852xo7BbtN34yDtfLv4sV5gXSQ9kg7InaxAv9yDH4lHacvHEvzoUoo2x89oJEBuU9j48yE9liHrVpQa1MPw9KkDlELItk5y8Zk3IUB8VhNCoJEDF2ve0Cf4EvmW8CMiER2yg8dl16lg12QQ3a0xGVSXD3OV4BQVBH6O9antX9UGMhMoWOnixONua2NHIU7LzQIWUvsiln5ZhmTg2uq3PdfkomEMKlUlQhA2umrvI2JO6yaY70-U1Oos9ESvhsr0lL03MMLsJP-_Xsd-BwBcBybZUpjVIM3xSgBVVVCcaPQZ_RNNjhAM06ZQsxDBhSdkqJWwWqRahcu07yM_2Rq_FjBddXS1eFlUX86zz_nBe7-MtvOT-AZx4OHQlB1xg=&p=0&fvj=0&vjs=3
https://www.indeed.com/rc/clk?jk=d6a941ebc428cbd8&fccid=537661dec74f1516&vjs=3
https://www.indeed.com/rc/clk?jk=ad8d8262642f8798&fccid=aaa2b906602aa8f5&vjs=3
https://www.indeed.com/pagead/clk?mo=r&ad=-6NYlbfkN0C1guQf8RKr4iIKXbDNY9cwFNNLA2MxixwVTgjDUWubC7x1HSBiYcZ4H4ttfk5q_GKCW2Ph5wIzknuJDoIJAQtekAYGaZNGWwJYFEa64glYkK1bn

In [ ]:
c =0
for data in df.iterrows():
  if( pd.isna(data[1]['full_desc'])):
    # print(data[0])
    c +=1
print(c)

0


In [ ]:
df.to_csv("/content/Robo_jobs_USA (few full_desc filled).csv")

In [ ]:
df = pd.read_csv('/content/Robo_jobs_USA (few full_desc filled).csv')

In [ ]:
def extract_full_desc(html):
  jd_card, _ = collect_job_cards_from_page(html,'div',  "jobsearch-jobDescriptionText")
  full_jd = get_parsed_full_desc(jd_card)
  return full_jd
tc = 0
c = 0
for data in df.iterrows():
  tc += 1
  if( pd.isna(data[1]['full_desc'])): #data[1]['full_desc'] == "" or data[1]['full_desc'] == " " or
    c += 1
    html = request_jobs_from_indeed(data[1]['JobUrl'])
    if(html == None):
      continue
    full_desc = extract_full_desc(html)
    df.iloc[data[0]]["full_desc"] = full_desc
    print(data[1]['JobUrl'])
print(tc,c)

len of p_ul_elements was zero, it is printted below
[]
https://www.indeed.com/pagead/clk?mo=r&ad=-6NYlbfkN0C3j_zLGvpMLCdiZ0WC46XqVTA1VMZzOzKXPhAXwYlrNag_LLRX9rPXJmTR3OCtRpZObNSVPMdPef_WLKyECbRz5Rz72ZYnBFXmAnVaAaNuuXf72ShGcjj3get4X1JFY2odjVf5JP_Vet_v1ReyMeVVO25sPH0wU2VTVz_7q9_SAFFmV-lTrDR0Lvs_LDIjFn3uWxDzfXilG4wHy4OSGMRBoXZ4q6Arl5NYRI9Kv-Y3o9FrzrIBvOAXfMqIo_D4SZfhuiZemxASgIGBOBzt1zL2pgSKCJuir3f-ifJ-MXByH6JKuB60dHTBUKAM0aW7veczyDyTQgqvZjBkG1v2xn-LTGUl5mVdkluf5xcLC51-r8fR1H7y3ZXQH72p_HTCuBgMtUh3WZ6RLsJZba3UWC2WAB9AxdpkQpc=&p=0&fvj=0&vjs=3
len of p_ul_elements was zero, it is printted below
[]
https://www.indeed.com/rc/clk?jk=5fa09511648e3203&fccid=c4e5eba52d49b54e&vjs=3
len of p_ul_elements was zero, it is printted below
[]
https://www.indeed.com/rc/clk?jk=eeeafc6644bc1c8d&fccid=2c23f29fcd5c78da&vjs=3


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app


len of p_ul_elements was zero, it is printted below
[]
https://www.indeed.com/rc/clk?jk=def05451632bd2d7&fccid=5b2f9eaa91cb3370&vjs=3
len of p_ul_elements was zero, it is printted below
[]
https://www.indeed.com/rc/clk?jk=4dbc3dafb6290c70&fccid=afbf8c270610a38a&vjs=3
len of p_ul_elements was zero, it is printted below
[]
https://www.indeed.com/rc/clk?jk=0f6e2fa5539d0e53&fccid=b1330bfa6e5aae73&vjs=3
len of p_ul_elements was zero, it is printted below
[]
https://www.indeed.com/rc/clk?jk=6d977b055ad4e96e&fccid=d55ba0bc1050147f&vjs=3
len of p_ul_elements was zero, it is printted below
[]
https://www.indeed.com/company/National-Geospatial--Intelligence-Agency/jobs/R&D-Scientist-80e9a9d41db7cfeb?fccid=5328ab5d4655b9b1&vjs=3
len of p_ul_elements was zero, it is printted below
[]
https://www.indeed.com/rc/clk?jk=2fa8d16bdd0fcd3a&fccid=734cb5a01ee60f80&vjs=3
len of p_ul_elements was zero, it is printted below
[]
https://www.indeed.com/rc/clk?jk=d85c429750d0510c&fccid=25d3835187829237&vjs=3
len 

In [ ]:
c1 =0;
for data in df.iterrows():
  if(pd.isna(data[1]['full_desc'])):
    pass
  else:
    c1+=1
    # print(data[1]['full_desc'])

print(c1)

238


In [ ]:
df.to_csv("/content/NLP_jobs_USA (few full_desc filled).csv")

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (0,3,12,34,35,36,37,38) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [ ]:
df.head()

In [ ]:
df1 = df["Unnamed: 36"]

In [ ]:
df1[2]

'https://app.dimensions.ai/details/publication/pub.1117374772'

# Credits and Acknoledgements

israel-dryer
https://github.com/israel-dryer/Indeed-Job-Scraper/blob/master/indeed_job_scraper.py

In [ ]:
import os

import json

from urllib.request import urlretrieve

from collections import defaultdict

import numpy as np

from math import log



files_to_download = {"CV.xlsx":"https://github.com/moyeed/corelation_between_jobs_and_research_papers/raw/main/Computer%20Vision.xlsx"}



for fileName, url in files_to_download.items():

  if not os.path.exists(fileName):

    urlretrieve(url, fileName)

In [ ]:
https://github.com/moyeed/corelation_between_jobs_and_research_papers/raw/main/Computer%20Vision.xlsx


Computer Vision.xlsx

In [ ]:
df1 = pd.read_excel("/content/CV.xlsx")


In [ ]:
df1.columns = df1.iloc[0]
df1 = df1.drop(0)

In [ ]:
df1

,Rank,Publication ID,DOI,PMID,PMCID,Title,Abstract,Acknowledgements,Source title,Anthology title,Publisher,MeSH terms,PubYear,Publication Date (online),Publication Date (print),Volume,Issue,Pagination,Open Access,Publication Type,Authors,Authors (Raw Affiliation),Corresponding Author,Authors Affiliations,Research Organizations - standardized,GRID IDs,City of Research organization,State of Research organization,Country of Research organization,Funder,Funder Group,Funder Country,UIDs of supporting grants,Supporting Grants,Times cited,Recent citations,RCR,FCR,Altmetric,Source Linkout,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,382,pub.1139882566,NaN,34290902,NaN,Can Deep Learning Recognize Subtle Human Activ...,Deep Learning has driven recent and exciting p...,NaN,2008 IEEE Computer Society Conference on Compu...,NaN,NaN,NaN,2020,2020-08-05 00:00:00,2020-06,2020,NaN,NaN,Closed,Article,"Jacquot, Vincent; Ying, Zhuofan; Kreiman, Gabriel","Jacquot, Vincent (Ã‰cole Polytechnique FÃ©dÃ©r...",NaN,"Jacquot, Vincent (Swiss Federal Institute of T...",University of Science and Technology of China;...,grid.59053.3a; grid.5333.6,Hefei; Lausanne,;,China; Switzerland,National Eye Institute,ICRP - International Cancer Research Partnersh...,United States,grant.4898766,R01EY026025,1,1,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,346,pub.1126910595,10.1155/2020/2561703,32372891,PMC7195649,Knowledge about Computer Vision Syndrome among...,"Background: Globally, computer vision syndrome...",NaN,Occupational Therapy International,NaN,Hindawi,Adult; Computer Terminals; Cross-Sectional Stu...,2020,2020-04-14 00:00:00,2020-04-23 00:00:00,2020,NaN,2561703,All OA; Gold,Article,"Mersha, Getasew Alemu; Hussen, Mohammed Seid; ...","Mersha, Getasew Alemu (Department of Optometry...","Mersha, Getasew Alemu (University of Gondar)","Mersha, Getasew Alemu (University of Gondar); ...",University of Gondar,grid.59547.3a,Gonder,NaN,Ethiopia,NaN,NaN,NaN,NaN,NaN,2,2,NaN,NaN,NaN,https://doi.org/10.1155/2020/2561703,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,319,pub.1124411906,10.1364/ol.384970,32163975,NaN,Metasurface-based contact lenses for color vis...,"We embed large-scale, plasmonic metasurfaces i...",NaN,Optics Letters,NaN,The Optical Society,"Coated Materials, Biocompatible; Color Percept...",2020,2020-03-05 00:00:00,2020-03-15 00:00:00,45,6,1379-1382,Closed,Article,"Karepov, Sharon; Ellenbogen, Tal","Karepov, Sharon (); Ellenbogen, Tal ()",NaN,"Karepov, Sharon (); Ellenbogen, Tal ()",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5,5,NaN,NaN,340,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,314,pub.1126628864,10.1128/jcm.00511-20,32295889,PMC7269399,Computer Vision and Artificial Intelligence Ar...,Artificial intelligence (AI) is increasingly b...,NaN,Journal of Clinical Microbiology,NaN,American Society for Microbiology,Artificial Intelligence; Clinical Laboratory S...,2020,2020-05-26 00:00:00,2020-05-26 00:00:00,58,6,NaN,All OA; Green,Article,"Rhoads, Daniel D","Rhoads, Daniel D (Department of Pathology, Cas...",NaN,"Rhoads, Daniel D (Case Western Reserve Univers...",Case Western Reserve University,grid.67105.35,Cleveland,Ohio,United States,NaN,NaN,NaN,NaN,NaN,5,5,NaN,NaN,11,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC7...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,313,pub.1112864062,10.1364/josaa.36.000606,31044981,NaN,Hyperspectral imaging in col